In [ ]:
# ==========================================================
# Importando as bibliotecas necessárias
# ==========================================================
import requests
from pyspark.sql import SparkSession  # Para criar e manipular DataFrames no Spark
import pandas as pd 

# Criando a sessão Spark
#spark = SparkSession.builder \
#    .appName("API REST Countries") \
#    .getOrCreate()
#print("SparkSession criada com sucesso!")

# ==========================================================
# Fazendo a requisição para a API REST Countries
# ==========================================================
url = "https://restcountries.com/v3.1/all"

# Requisição GET para obter os dados dos países
response = requests.get(url)

# Verificamos se a resposta foi bem-sucedida (código 200)
if response.status_code == 200:
    print("Conexão bem-sucedida com a API REST Countries!")

    # Convertendo a resposta em formato JSON
    paises = response.json()

    # Lista para armazenar os dados dos países
    lista_paises = []

    # ==========================================================
    # Extraindo os dados desejados de cada país
    # ==========================================================
    for pais in paises:
        # Extraindo o nome comum do país
        nome = pais.get('name', {}).get('common', 'Não informado')

        # Extraindo o código alpha-2 (duas letras)
        codigo_alpha2 = pais.get('cca2', 'Não informado')

        # Extraindo o código alpha-3 (três letras)
        codigo_alpha3 = pais.get('cca3', 'Não informado')

        # Extraindo o código numérico
        codigo_numerico = pais.get('ccn3', 'Não informado')

        # Extraindo o código olímpico, se existir
        codigo_olimpico = pais.get('cioc', 'Não informado')

        # Extraindo o nome oficial do país
        official_name = pais.get("name", {}).get("official")

        # Adicionando o país na lista como dicionário
        lista_paises.append({
            'nome': nome,
            'codigo_alpha2': codigo_alpha2,
            'codigo_alpha3': codigo_alpha3,
            'codigo_numerico': codigo_numerico,
            'codigo_olimpico': codigo_olimpico,
            'official_name': official_name
        })

    print("Dados dos países coletados com sucesso!")

else:
    print(f"Erro na requisição. Código de status: {response.status_code}")



# ==========================================================
#  Criando o DataFrame Pandas com os dados coletados
# ==========================================================

    # Definimos os nomes das colunas para o DataFrame 

colunas = ["nome", "codigo_alpha2", "codigo_alpha3", "codigo_numerico", "codigo_olimpico","official_name"]

df_lista_paises = pd.DataFrame(lista_paises, columns=colunas)

print("DataFrame criado com sucesso!")


# ==========================================================
# Exibindo o DataFrame para visualização 
# ==========================================================

display(df_lista_paises)  # Comando para visualizar em formato de tabela

# ==========================================================
# Criando uma Temp View para consultas SQL 
# ==========================================================

#df_lista_paises.createOrReplaceGlobalTempView("view_paises_codigos")
#print("✅ Temp View 'view_paises_codigos' criada com sucesso!")


Conexão bem-sucedida com a API REST Countries!
Dados dos países coletados com sucesso!
DataFrame criado com sucesso!


,nome,codigo_alpha2,codigo_alpha3,codigo_numerico,codigo_olimpico,official_name
0,Botswana,BW,BWA,072,BOT,Republic of Botswana
1,Tonga,TO,TON,776,TGA,Kingdom of Tonga
2,Greece,GR,GRC,300,GRE,Hellenic Republic
3,Marshall Islands,MH,MHL,584,MHL,Republic of the Marshall Islands
4,Belarus,BY,BLR,112,BLR,Republic of Belarus
...,...,...,...,...,...,...
245,Bahamas,BS,BHS,044,BAH,Commonwealth of the Bahamas
246,Eswatini,SZ,SWZ,748,SWZ,Kingdom of Eswatini
247,Brazil,BR,BRA,076,BRA,Federative Republic of Brazil
248,Lithuania,LT,LTU,440,LTU,Republic of Lithuania


In [17]:
from datetime import datetime
import pytz

fuso_brasil = pytz.timezone("America/Sao_Paulo")
agora_brasil = datetime.now(fuso_brasil)

df_lista_paises['data_carga'] = agora_brasil.strftime('%Y-%m-%d %H:%M:%S')


In [6]:
#!pip install pandasql
from pandasql import sqldf

# Define uma função para executar consultas SQL
pysqldf = lambda q: sqldf(q, globals())

In [19]:
consultaSQL = """

SELECT *
FROM df_lista_paises 
--where nome = "Brazil"

"""

resultado = pysqldf(consultaSQL)
display(resultado)

,nome,codigo_alpha2,codigo_alpha3,codigo_numerico,codigo_olimpico,official_name,data_carga
0,Botswana,BW,BWA,072,BOT,Republic of Botswana,2025-05-03 12:59:48
1,Tonga,TO,TON,776,TGA,Kingdom of Tonga,2025-05-03 12:59:48
2,Greece,GR,GRC,300,GRE,Hellenic Republic,2025-05-03 12:59:48
3,Marshall Islands,MH,MHL,584,MHL,Republic of the Marshall Islands,2025-05-03 12:59:48
4,Belarus,BY,BLR,112,BLR,Republic of Belarus,2025-05-03 12:59:48
...,...,...,...,...,...,...,...
245,Bahamas,BS,BHS,044,BAH,Commonwealth of the Bahamas,2025-05-03 12:59:48
246,Eswatini,SZ,SWZ,748,SWZ,Kingdom of Eswatini,2025-05-03 12:59:48
247,Brazil,BR,BRA,076,BRA,Federative Republic of Brazil,2025-05-03 12:59:48
248,Lithuania,LT,LTU,440,LTU,Republic of Lithuania,2025-05-03 12:59:48


In [20]:
df_export = pd.DataFrame(resultado)  # Dados dim pais
df_export.to_csv("df_dim_pais.csv", index=False)


In [22]:
import psycopg2

# Parâmetros de conexão
dbname   = 'projeto_paises'
user     = 'postgres'
password = '252500'
host     = 'localhost'
port     = '5432' 

# Criar uma conexão
conn = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)


cur = conn.cursor()# Criar um cursor  deixa manipular os dados
cur.execute("DELETE FROM TBL_DIM_PAIS")

for indice, colunas_df in resultado.iterrows():
        cur.execute('''                                
                insert into TBL_DIM_PAIS (   
                 nome,
                 codigo_alpha2,
                 codigo_alpha3,
                 codigo_numerico,
                 codigo_olimpico,
                 official_name,
                 data_carga)
                 VALUES (%s,%s,%s,%s,%s,%s,%s)
                        ''',
        (
        colunas_df["nome"],
        colunas_df["codigo_alpha2"],
        colunas_df["codigo_alpha3"],
        colunas_df["codigo_numerico"],
        colunas_df["codigo_olimpico"],
        colunas_df["official_name"],
        colunas_df["data_carga"]
        )
        )

#comandos sql transct-sql
conn.commit() # validar alterações que agente fez  e subir para o banco de dados 

# Fechar o cursor e a conexão
cur.close()
conn.close()